In [1]:
from sklearn.linear_model import LogisticRegression
from preprocessor import Preprocessor
from evaluator import ModelEvaluator
import pandas as pd
import numpy as np

In [2]:
'''
woe обработка категориальных фичей
'''

'\nwoe обработка категориальных фичей\n'

In [3]:
df_train = pd.read_csv("C:/Users/Роман/Chepenkov_dz2/application_train.csv")
df_test = pd.read_csv("C:/Users/Роман/Chepenkov_dz2/application_test.csv")

In [4]:
WOE_MAPPING = {}

def hypothesis_8(df):
    
    min_samples=100
    alpha=0.5
    TARGET_SERIES = df_train['TARGET']
    global WOE_MAPPING
    
    df = df.copy()
    cat_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
    
    # Режим обучения (когда установлена целевая переменная)
    if TARGET_SERIES is not None and len(WOE_MAPPING) == 0:
        for col in cat_cols:
            # Объединяем данные для анализа
            temp_df = pd.DataFrame({col: df[col], 'target': TARGET_SERIES})
            
            # Вычисляем WOE
            freq_table = pd.crosstab(temp_df[col], temp_df['target'], margins=True)
            freq_table['non_event'] = freq_table[0] + alpha
            freq_table['event'] = freq_table[1] + alpha
            freq_table['distr_non_event'] = freq_table['non_event'] / freq_table.at['All', 'non_event']
            freq_table['distr_event'] = freq_table['event'] / freq_table.at['All', 'event']
            freq_table['woe'] = np.log(freq_table['distr_non_event'] / freq_table['distr_event'])
            
            # Сохраняем mapping
            WOE_MAPPING[col] = freq_table['woe'].to_dict()
            del WOE_MAPPING[col]['All']
            
            # Применяем к данным
            df[col] = df[col].map(WOE_MAPPING[col]).fillna(freq_table.at['All', 'woe'])
    
    # Режим применения на тесте
    elif len(WOE_MAPPING) > 0:
        for col in cat_cols:
            if col in WOE_MAPPING:
                df[col] = df[col].map(WOE_MAPPING[col])
                # Заполняем новые категории средним WOE
                if df[col].isna().any():
                    overall_woe = np.mean(list(WOE_MAPPING[col].values()))
                    df[col] = df[col].fillna(overall_woe)
    
    return df

In [5]:
preprocessor = Preprocessor(custom_steps=[hypothesis_8])

X_train_processed = preprocessor.transform(df_train.drop(columns=['TARGET', 'SK_ID_CURR']))

X_test_processed = preprocessor.transform(df_test.drop(columns=['SK_ID_CURR']))

In [6]:
evaluator = ModelEvaluator(LogisticRegression(max_iter=100))

roc_auc = evaluator.evaluate(X_train_processed, df_train['TARGET'])
print(f"ROC-AUC train: {roc_auc:.5f}")

evaluator.save_test_predictions(
    X_test=X_test_processed,
    ids=df_test['SK_ID_CURR'],
    filename='test_predictions_hyp_8.csv'
)
#ROC-AUC train: 0.61968

#ROC-AUC test: 0.60998
#ROC-AUC test baseline: 0.61204
#ROC-AUC test surplus: -0.00206

C:\Users\Роман\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


ROC-AUC train: 0.61968
